In [1]:
# Read files
twitterRDD = sc.textFile("C:/Users/byron/Documents/UMN/MSBA 6330 Harvesting Big Data/Project/data_big/data_big_text.csv").cache()

twitterRDD_split = twitterRDD.map(lambda line: line.split(",")).cache()
twitterRDD_split.take(5)

[[u'Tweet content'],
 [u'RT @DILLONFRANCIS: Dude @CalvinHarris fucking amazing!! https://t.co/pbNZr86ZQi'],
 [u'Apple iPod classic 7th Generation Black (160 GB) https://t.co/RA5Rw9cbbB https://t.co/MaOcoRfnjJ'],
 [u'"Apple iPhone 6 - 16GB - Gold (Sprint) Clean Serial Number',
  u' Has 1 issue',
  u' read des - Bid Now\ufffd https://t.co/ewrYsGJQSx https://t.co/yJzJFApW6E"'],
 [u'Get Jokey Smurfs brand smurfin new hut to play his Stocking Stuffer mini game. #SmurfsVillage https://t.co/EcuBV0OaVV']]

In [2]:
# Function to clean words
def cleanWords(word):
    new_word = str(word).lower().replace("u'",'').replace('[', '').replace(']', '').replace(',', '').replace("'", '')\
        .replace('"','').replace("(",'').replace(")",'').replace("!",'').replace("?",'').replace(":",'')   
    if new_word != '':
        return new_word
    else:
        return 'NA'

In [3]:
twitterRDD_cleaned = twitterRDD_split.map(lambda line: cleanWords(line))

In [4]:
# import positive and negative words
import re
positiveWords = open('C:/Users/byron/Documents/UMN/MSBA 6330 Harvesting Big Data/Project/data_big/positive-words.txt', 'r')
negativeWords = open('C:/Users/byron/Documents/UMN/MSBA 6330 Harvesting Big Data/Project/data_big/negative-words.txt', 'r')
posWords = re.split(r'\n', positiveWords.read())
negWords = re.split(r'\n', negativeWords.read())

# This files and the papers of positive and negative words can all be downloaded from 
#    http://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html
#
# Sources:
#
#   Minqing Hu and Bing Liu. "Mining and Summarizing Customer Reviews." 
#       Proceedings of the ACM SIGKDD International Conference on Knowledge 
#       Discovery and Data Mining (KDD-2004), Aug 22-25, 2004, Seattle, 
#       Washington, USA, 
#   Bing Liu, Minqing Hu and Junsheng Cheng. "Opinion Observer: Analyzing 
#       and Comparing Opinions on the Web." Proceedings of the 14th 
#       International World Wide Web conference (WWW-2005), May 10-14, 
#       2005, Chiba, Japan.

In [5]:
# Function for sentiment analysis
def lookup_for(line):
    word_list = str(line).split()
    word_count = len(word_list)
    positive_words = 0
    negative_words = 0
    #offset of words you want to look for 
    for word in word_list: 
        if word.lower() in posWords:
            positive_words +=1
        if word.lower() in negWords:
            negative_words +=1
    score = float(positive_words - negative_words) / word_count
            
    return score

In [6]:
# Calculate sentiment
twitterScoresRDD = twitterRDD_cleaned.map(lookup_for).cache()
twitterScoresRDD.take(5)

[0.0, 0.0, 0.1, 0.047619047619047616, 0.0]

In [7]:
twitterScoresRDD.count()

118356

In [8]:
twitterScoresRDD.max()

0.5

In [9]:
twitterScoresRDD.min()

-0.6666666666666666

In [10]:
twitterScoresRDD.mean()

0.016272906862605675

In [11]:
# Save result in csv file
#strRDD = twitterScoresRDD.map(lambda a : str(a) + ",")
#strRDD.saveAsTextFile('C:/Users/byron/Documents/UMN/MSBA 6330 Harvesting Big Data/Project/data_big/sentiment_scores_new.csv')

In [12]:
# Verify last 15 rows
twitterScoresRDD.zipWithIndex().filter(lambda tup: tup[1] > 118330).collect()

[(0.0, 118331),
 (0.0, 118332),
 (0.0, 118333),
 (0.0, 118334),
 (0.0625, 118335),
 (0.0, 118336),
 (0.0, 118337),
 (0.0, 118338),
 (0.0, 118339),
 (0.058823529411764705, 118340),
 (0.0, 118341),
 (0.0, 118342),
 (0.0, 118343),
 (0.0, 118344),
 (0.0, 118345),
 (0.0, 118346),
 (0.07142857142857142, 118347),
 (0.0, 118348),
 (0.0, 118349),
 (0.0, 118350),
 (0.08333333333333333, 118351),
 (-0.21052631578947367, 118352),
 (0.0, 118353),
 (0.0, 118354),
 (0.1, 118355)]

In [18]:
# Get special topics for analysis
music = ['music', 'music.']
iphone = ['iphone']
ipad = ['ipad', 'ipod']
drake = ['drake', '@drake']
itunes = ['itunes']

def lookup_for_topic(line, alist):
    word_list = str(line).split()
    has_word = 0
    for word in word_list: 
        if word.lower() in alist:
            has_word = 1
    
    return has_word

musicRDD = twitterRDD_cleaned.map(lambda line: lookup_for_topic(line, music)).map(lambda a : str(a) + ",")
iphoneRDD = twitterRDD_cleaned.map(lambda line: lookup_for_topic(line, iphone)).map(lambda a : str(a) + ",")
ipadRDD = twitterRDD_cleaned.map(lambda line: lookup_for_topic(line, ipad)).map(lambda a : str(a) + ",")
drakeRDD = twitterRDD_cleaned.map(lambda line: lookup_for_topic(line, drake)).map(lambda a : str(a) + ",")
itunesRDD = twitterRDD_cleaned.map(lambda line: lookup_for_topic(line, itunes)).map(lambda a : str(a) + ",")

#musicRDD.saveAsTextFile('C:/Users/byron/Documents/UMN/MSBA 6330 Harvesting Big Data/Project/data_big/aboutMusic.csv')
#iphoneRDD.saveAsTextFile('C:/Users/byron/Documents/UMN/MSBA 6330 Harvesting Big Data/Project/data_big/aboutiPhone.csv')
#ipadRDD.saveAsTextFile('C:/Users/byron/Documents/UMN/MSBA 6330 Harvesting Big Data/Project/data_big/aboutiPad.csv')
#drakeRDD.saveAsTextFile('C:/Users/byron/Documents/UMN/MSBA 6330 Harvesting Big Data/Project/data_big/aboutDrake.csv')
#itunesRDD.saveAsTextFile('C:/Users/byron/Documents/UMN/MSBA 6330 Harvesting Big Data/Project/data_big/aboutiTune.csv')